In [2]:
import cv2
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import os
from sklearn.metrics import accuracy_score

# Проверка доступности GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda:0


In [3]:
# Задаем размер входного изображения
input_size = 48  

# Трансформации для тренировочного набора данных 
data_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),  # Изменение размера
    transforms.ToTensor(),  # Преобразование в тензор
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Нормализация
])

# Трансформации для валидации 
val_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),  # Изменение размера
    transforms.ToTensor(),  # Преобразование в тензор
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Нормализация
])

# Загрузка данных из разных папок
train_dir = r"C:\Users\Omen\Desktop\Datasets_NEW\train"  # Папка с тренировочными данными
val_dir = r"C:\Users\Omen\Desktop\Datasets_NEW\test"  # Папка с валидационными данными

# Создаем наборы данных с использованием трансформаций
train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transforms)

# DataLoader для загрузки данных в батчах
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Словарь для DataLoader'ов и размеров наборов данных
dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
class_names = train_dataset.classes  # Можно использовать только тренировочные классы, если они одинаковы

print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")
print(f"Classes: {class_names}")

Train dataset size: 73283
Val dataset size: 7665
Classes: ['angry', 'fearful', 'happy', 'neutral', 'sad', 'surprised']


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EmotionRecognitionCNN(nn.Module):
    def __init__(self, num_classes):
        super(EmotionRecognitionCNN, self).__init__()

        self.features = nn.Sequential(
            # Первый блок сверток
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # 3 канала для цветных изображений
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Уменьшение размерности

            # Второй блок сверток
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Уменьшение размерности

            # Третий блок сверток
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Уменьшение размерности
        )

        # Классификатор
        self.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 1024),  # Входной размер: 256 каналов после сверток
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)  # Выходной слой с числом классов
        )

    def forward(self, x):
        x = self.features(x)  # Применение сверток
        x = x.view(x.size(0), -1)  # Разворачиваем тензор для подачи в fully connected слои
        x = self.classifier(x)  # Применение классификатора
        return x




# Инициализируем модель
num_classes = len(class_names)
model = EmotionRecognitionCNN(num_classes=num_classes)
model = model.to(device)

# Определим функцию потерь и оптимизатор с L2-регуляризацией
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)  # Добавляем L2-регуляризацию через weight_decay

# Планировщик для уменьшения скорости обучения при остановке улучшений
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [5]:
# Реализация ранней остановки
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_loss = float('inf')

    def __call__(self, val_loss, model, model_save_path):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, model_save_path)
        elif score < self.best_score:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, model_save_path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, model_save_path):
        '''Сохранение модели, если валидационная ошибка уменьшилась'''
        if self.verbose:
            print(f'Validation loss decreased ({self.best_loss:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), model_save_path)
        self.best_loss = val_loss


In [5]:
import time
import torch
from tqdm import tqdm
import time

# Функция обучения модели
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, model_save_path='best_model.pth', patience=5):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0
    early_stopping = EarlyStopping(patience=patience, verbose=True)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        # Каждая эпоха содержит этап обучения и валидации
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Установить модель в режим тренировки
            else:
                model.eval()   # Установить модель в режим оценки

            running_loss = 0.0
            running_corrects = 0

            # Получаем данные
            dataloader = dataloaders[phase]
            dataset_size = dataset_sizes[phase]

            # Используем tqdm для отслеживания прогресса
            for inputs, labels in tqdm(dataloader, desc=f"{phase.capitalize()} Progress", leave=False):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Обнуляем градиенты
                optimizer.zero_grad()

                # Прямой проход
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Обратный проход + оптимизация только на этапе обучения
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Статистика
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Сохраняем модель при улучшении на валидации
            if phase == 'val':
                early_stopping(epoch_loss, model, model_save_path)
                if early_stopping.early_stop:
                    print("Early stopping triggered")
                    time_elapsed = time.time() - since
                    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
                    
                    # Загружаем наилучшие веса модели
                    model.load_state_dict(torch.load(model_save_path))
                    return model

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    # Загружаем наилучшие веса модели
    model.load_state_dict(torch.load(model_save_path))
    return model

In [6]:
# Обучаем модель с ранней остановкой и сохранением лучших весов
model_save_path = 'emotion_recognition_model_V4_2.pth'
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=30, model_save_path=model_save_path, patience=5)
torch.save(model.state_dict(), model_save_path)

Epoch 1/30
----------


train Loss: 1.6790 Acc: 0.2778


val Loss: 1.4075 Acc: 0.4457
Validation loss decreased (inf --> 1.407539).  Saving model ...
Epoch 2/30
----------


train Loss: 1.3674 Acc: 0.4505


val Loss: 1.1825 Acc: 0.5536
Validation loss decreased (1.407539 --> 1.182496).  Saving model ...
Epoch 3/30
----------


...

train Loss: 0.4722 Acc: 0.8288


val Loss: 0.3871 Acc: 0.8934
Validation loss decreased (0.388559 --> 0.387051).  Saving model ...
Epoch 24/30
----------


train Loss: 0.4722 Acc: 0.8287


val Loss: 0.3855 Acc: 0.8935
Validation loss decreased (0.387051 --> 0.385522).  Saving model ...
Epoch 25/30
----------


KeyboardInterrupt: 

In [6]:
import cv2
import torch
import torch.nn.functional as F
import numpy as np
from torchvision import transforms

# Загрузка обученной модели
model = EmotionRecognitionCNN(num_classes=6)
model.load_state_dict(torch.load('emotion_recognition_model_V4_2.pth'))
model.eval()

# Загрузка классификатора Хаара для обнаружения лиц
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Преобразования для предобработки изображения
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Используем три канала
])

# Метки эмоций
emotions = ['Angry', 'Fearful', 'Happy', 'Sad', 'Surprised', 'Neutral']

# Запуск камеры
cap = cv2.VideoCapture(0)

while True:
    # Захват изображения с камеры
    ret, frame = cap.read()
    if not ret:
        break

    # Обнаружение лиц на изображении
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Извлечение лица
        face = frame[y:y + h, x:x + w]

        # Преобразование для модели
        face_img = transform(face).unsqueeze(0)  # Добавляем размер батча

        # Предсказание эмоции
        with torch.no_grad():
            output = model(face_img)
            emotion_prediction = F.softmax(output, dim=1)
            emotion_label = torch.argmax(emotion_prediction).item()

        # Метка предсказанной эмоции
        emotion_text = emotions[emotion_label]

        # Отображение предсказанной эмоции на изображении
        cv2.putText(frame, emotion_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Показ изображения
    cv2.imshow('Emotion Recognition', frame)

    # Прерывание по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение камеры и закрытие окон
cap.release()
cv2.destroyAllWindows()



C:\Users\Omen\AppData\Local\Temp\ipykernel_21988\3775270898.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('emotion_recognition_model_V

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Выбираем GPU, если доступно

# Переносим модель на устройство
model = model.to(device)

# Функция для расчета точности
def evaluate_model(model, val_loader):
    model.eval()  # Устанавливаем режим оценки (отключает dropout и batchnorm)
    correct = 0
    total = 0

    with torch.no_grad():  # Отключаем градиенты для ускорения вычислений
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)  # Переносим данные на устройство (GPU/CPU)
            outputs = model(images)  # Получаем предсказания от модели
            _, predicted = torch.max(outputs.data, 1)  # Выбираем метку с наибольшей вероятностью
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  # Сравниваем предсказания с истинными метками

    accuracy = 100 * correct / total  # Расчет точности в процентах
    print(f'Test Accuracy: {accuracy:.2f}%')

# Вызов функции для проверки точности
evaluate_model(model, val_loader)


Test Accuracy: 89.35%
